In [1]:
#export
settings = {"defaultDelim": "\t", "defaultIndent": "  ",
            "oboFile": None, "strict": False, "lookupImgs": True}
def patchDefaultDelim(s:str):
    """
:param s:
    - if not None, returns self
    - else returns the default delimiter in settings"""
    return settings["defaultDelim"] if s is None else s
def patchDefaultIndent(s:str):
    """
:param s:
    - if not None, returns self
    - else returns the default indent character in settings"""
    return settings["defaultIndent"] if s is None else s
from typing import List, Iterator, Any, NewType, TypeVar, Generic
import k1lib.cli as cli; from numbers import Number
import itertools, copy, torch; import numpy as np

__all__ = ["BaseCli", "serial", "oneToMany", "manyToMany", "manyToManySpecific", "Table", "T"]

In [2]:
#export
T = TypeVar("T")
"""Generic type variable"""
class _MetaType(type):
    def __getitem__(self, generic):
        d = {"__args__": generic, "_n": self._n, "__doc__": self.__doc__}
        return _MetaType(self._n, (), d)
    def __repr__(self):
        def main(self):
            def trueName(o):
                if isinstance(o, _MetaType): return main(o)
                try: return o.__name__
                except: return f"{o}"
            if hasattr(self, "__args__"):
                if isinstance(self.__args__, tuple):
                    return f"{self._n}[{', '.join([trueName(e) for e in self.__args__])}]"
                else: return f"{self._n}[{trueName(self.__args__)}]"
            return self._n
        return main(self)
def newTypeHint(name, docs=""):
    """Creates a new type hint that can be sliced and yet still looks fine
in sphinx. Crudely written by my poorly understood idea of Python's
metaclasses. Seriously, this shit is bonkers, read over it https://stackoverflow.com/questions/100003/what-are-metaclasses-in-python

Example::

    Table = newTypeHint("Table", "some docs")
    Table[int] # prints out as "Table[int]", and sphinx fell for it too
    Table[Table[str], float] # prints out as "Table[Table[str], float]"
"""
    return _MetaType(name, (), {"_n": name, "__doc__": docs})
#Table = newTypeHint("Table", """Essentially just Iterator[List[T]]. This class is just here so that I can generate the docs with nicely formatted types like "Table[str]".""")
#Table = NewType("Table", List)
class Table(Generic[T]):
    """Essentially just Iterator[List[T]]. This class is just here so that I can generate the docs with nicely formatted types like "Table[str]"."""
    pass
Table._name = "Table"
#Table.__module__ = "cli"
class Row(list):
    """Not really used currently. Just here for potential future feature"""
    pass

In [3]:
#export
class BaseCli:
    """A base class for all the cli stuff. You can definitely create new cli tools that
have the same feel without extending from this class, but advanced stream operations
(like ``+``, ``&``, ``.all()``, ``|``) won't work.

At the moment, you don't have to call super().__init__() and super().__ror__(),
as __init__'s only job right now is to solidify any :class:`~k1lib.cli.modifier.op`
passed to it, and __ror__ does nothing."""
    def __init__(self, fs=[]):
        """Not expected to be instantiated by the end user.

:param fs: if functions inside here is actually a :class:`~k1lib.cli.modifier.op`,
    then solidifies it (make it not absorb __call__ anymore)"""
        [f.op_solidify() for f in fs if isinstance(f, cli.op)]
    def __and__(self, cli:"BaseCli") -> "oneToMany":
        """Duplicates input stream to multiple joined clis."""
        if isinstance(self, oneToMany):
            self.clis.append(cli); return self
        if isinstance(cli, oneToMany):
            cli.clis.append(self); return cli
        return oneToMany(self, cli)
    def __add__(self, cli:"BaseCli") -> "manyToManySpecific":
        """Parallel pass multiple streams to multiple clis."""
        if isinstance(self, manyToManySpecific):
            self.clis.append(cli); return self
        if isinstance(cli, manyToManySpecific):
            cli.clis.append(self); return cli
        return manyToManySpecific(self, cli)
    def all(self) -> "BaseCli":
        """Applies this cli to all incoming streams"""
        return manyToMany(self)
    def __or__(self, it) -> "serial":
        """Joins clis end-to-end"""
        if isinstance(it, BaseCli) or hasattr(it, "__ror__"):
            return serial(self, it)
    def __ror__(self, it): return NotImplemented
    def f(self) -> Table[Table[int]]:
        """Creates a normal function :math:`f(x)` which is equivalent to
``x | self``."""
        return lambda it: self.__ror__(it)
    def __lt__(self, it):
        """Default backup join symbol `>`, in case `it` implements __ror__()"""
        return self.__ror__(it)
    def __call__(self, it, *args):
        """Another way to do ``it | cli``. If multiple arguments are fed, then the
argument list is passed to cli instead of just the first element. Example::

    @applyS
    def f(it):
        return it
    f(2) # returns 2
    f(2, 3) # returns [2, 3]"""
        if len(args) == 0: return self.__ror__(it)
        else: return self.__ror__([it, *args])
class serial(BaseCli):
    def __init__(self, *clis:List[BaseCli]):
        """Merges clis into 1, feeding end to end. Used in chaining clis
together without a prime iterator. Meaning, without this, stuff like this
fails to run::

    [1, 2] | a() | b() # runs
    c = a() | b(); [1, 2] | c # doesn't run if this class doesn't exist"""
        super().__init__(fs=clis); self.clis = clis
    def __ror__(self, it:Iterator[Any]) -> Iterator[Any]:
        super().__ror__(it); clis = iter(self.clis)
        for cli in clis: it = cli.__ror__(it)
        return it

In [4]:
@cli.applyS
def f(it): return it
assert f(2) == 2
assert f(2, 3) == [2, 3]

In [5]:
#export
class oneToMany(BaseCli):
    def __init__(self, *clis:List[BaseCli]):
        """Duplicates 1 stream into multiple streams, each for a cli in the
list. Used in the "a & b" joining operator"""
        super().__init__(fs=clis); self.clis = clis
    def __ror__(self, it:Iterator[Any]) -> Iterator[Iterator[Any]]:
        super().__ror__(it); notIterable = False
        try: iter(it)
        except: notIterable = True
        if notIterable or isinstance(it, (list, tuple, torch.Tensor, str, Number, np.number, bool)):
            for cli in self.clis: yield cli.__ror__(it)
        else:
            its = itertools.tee(it, len(self.clis))
            for cli, it in zip(self.clis, its): yield cli.__ror__(it)
class manyToMany(BaseCli):
    def __init__(self, cli:BaseCli):
        """Applies multiple streams to a single cli. Used in the "a.all()"
operator. Note that this operation will use a different copy of the
cli for each of the streams."""
        super().__init__(fs=[cli]); self.cli = cli
    def __ror__(self, it:Iterator[Iterator[Any]]) -> Iterator[Iterator[Any]]:
        super().__ror__(it); c = self.cli
        f = c.ab_operate if isinstance(c, cli.op) else c.__ror__
        return (f(s) for s in it)
class manyToManySpecific(BaseCli):
    def __init__(self, *clis:List[BaseCli]):
        """Applies multiple streams to multiple clis independently. Used in
the "a + b" joining operator """
        super().__init__(fs=clis); self.clis = list(clis)
    def __ror__(self, its:Iterator[Any]) -> Iterator[Any]:
        super().__ror__(its)
        for cli, it in zip(self.clis, its): yield cli.__ror__(it)

In [6]:
!../../export.py cli/init

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.7
Uninstalling k1lib-0.7:
  Successfully uninstalled k1lib-0.7
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/fmt.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib